реалізація простого чатбота

In [ ]:
import os
import difflib
import random
import tkinter as tk
from tkinter import scrolledtext

class ChatBot:
    def __init__(self, responses_file="responses.txt", log_file="chat_log.txt"):
        self.responses_file = responses_file
        self.log_file = log_file
        self.responses = self.load_responses()

    def load_responses(self):
        file_manager = FileManager(self.responses_file)
        if os.path.exists(self.responses_file):
            return file_manager.read_responses()
        else:
            return {
                "привіт": "Привіт!",
                "як справи": "Відмінно! Як я можу допомогти вам?"
            }

    def save_responses(self):
        file_manager = FileManager(self.responses_file)
        file_manager.write_responses(self.responses)

    def show_log(self):
        file_manager = FileManager(self.log_file)
        try:
            lines = file_manager.read_lines()
            last_messages = lines[-10:]
            log_content = "\n".join(last_messages)
            return log_content
        except FileNotFoundError:
            return "Лог-файл не знайдено."

    def save_to_log(self, message):
        file_manager = FileManager(self.log_file)
        file_manager.append_line(message)

    def get_closest_match(self, user_input):
        matches = difflib.get_close_matches(user_input, self.responses.keys(), n=1, cutoff=0.6)
        return matches[0] if matches else None

    def respond(self, user_input):
        user_input = user_input.lower()
        if user_input in self.responses:
            response = self.responses[user_input]
            if callable(response):
                response()
            else:
                self.save_to_log(f"Користувач: {user_input}\nБот: {response}")
                return response
        else:
            closest_match = self.get_closest_match(user_input)
            if closest_match:
                response = self.responses[closest_match]
                self.save_to_log(f"Користувач: {user_input} (опечатка)\nБот: {response}")
                return f"Вибачте, не розумію це питання. Можливо, ви мали на увазі: {closest_match}\nБот: {response}"
            else:
                self.save_to_log(f"Користувач: {user_input}\nБот: Вибачте, не розумію це питання.")
                return "Вибачте, не розумію це питання."

    def add_response(self, user_input, bot_response):
        self.responses[user_input.lower()] = bot_response
        self.save_responses()
        return "Нова відповідь додана!"

class FileManager:
    def __init__(self, file_path):
        self.file_path = file_path

    def read_lines(self):
        with open(self.file_path, "r", encoding="utf-8") as file:
            return file.readlines()

    def read_responses(self):
        lines = self.read_lines()
        responses = dict(map(str.strip, line.split(":", 1)) for line in lines)
        return responses

    def write_responses(self, responses):
        with open(self.file_path, "w", encoding="utf-8") as file:
            for user_input, bot_response in responses.items():
                file.write(f"{user_input}: {bot_response}\n")

    def append_line(self, line):
        with open(self.file_path, "a", encoding="utf-8") as file:
            file.write(line + "\n")

class ChatBotGUI:
    def __init__(self, master, chat_bot):
        self.master = master
        master.title("Chat Bot")

        self.chat_bot = chat_bot

        self.input_entry = tk.Entry(master, width=50)
        self.input_entry.grid(row=0, column=0, padx=10, pady=10)

        self.output_text = scrolledtext.ScrolledText(master, width=60, height=15)
        self.output_text.grid(row=1, column=0, padx=10, pady=10)

        self.send_button = tk.Button(master, text="Відправити", command=self.send_message)
        self.send_button.grid(row=2, column=0, padx=10, pady=10)

        self.log_button = tk.Button(master, text="Покажи лог", command=self.show_log)
        self.log_button.grid(row=3, column=0, padx=10, pady=10)

    def send_message(self):
        user_input = self.input_entry.get()
        self.input_entry.delete(0, tk.END)
        bot_response = self.chat_bot.respond(user_input)
        self.output_text.insert(tk.END, f"Ви: {user_input}\nБот: {bot_response}\n\n")
        self.output_text.see(tk.END)

    def show_log(self):
        log_content = self.chat_bot.show_log()
        self.output_text.insert(tk.END, f"=== Лог ===\n{log_content}\n")
        self.output_text.see(tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    chat_bot = ChatBot()
    chat_bot_gui = ChatBotGUI(root, chat_bot)
    root.mainloop()
